In [ ]:
!pip install opencv-python numpy pillow matplotlib faker
!pip install opencv-python-headless pillow faker qrcode moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
from faker import Faker
import textwrap
import os
import qrcode
from tqdm import tqdm

fake = Faker()

# Traits, hobbies, and quotes lists
TRAITS = [
    "outgoing and friendly", "reserved and analytical", "creative and spontaneous",
    "detail-oriented and disciplined", "adventurous and curious", "empathetic and kind",
    "humorous and light-hearted", "organized and goal-oriented", "imaginative and dreamy",
    "bold and fearless", "thoughtful and introspective", "charming and persuasive"
]

HOBBIES = [
    "reading books", "painting", "playing video games", "traveling",
    "photography", "cooking", "sports", "music and dancing",
    "writing blogs", "volunteering", "yoga and meditation",
    "gardening", "hiking", "learning new languages"
]

QUOTES = [
    "Be yourself; everyone else is already taken.",
    "Life is what happens when you're busy making other plans.",
    "Success is not the key to happiness. Happiness is the key to success.",
    "In a world where you can be anything, be kind."
]

def generate_character():
    """Generate a random character bio."""
    name = fake.name()
    occupation = fake.job()
    trait = random.choice(TRAITS)
    hobby = random.choice(HOBBIES)
    quote = random.choice(QUOTES)

    bio = (
        f"{name} is a {occupation}. They are known to be {trait}. "
        f"In their free time, they enjoy {hobby}.\n\nQuote: \"{quote}\""
    )
    return bio, name

def cartoonify_image(image_path, cartoon_style='basic'):
    """Apply a cartoon filter with additional options."""
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"Image not found: {image_path}")

        img = cv2.resize(img, (600, 600), interpolation=cv2.INTER_AREA)

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)

        if len(faces) > 0:
            x, y, w, h = faces[0]
            img = img[y:y+h, x:x+w]

        # Apply different cartoon styles
        if cartoon_style == 'basic':
            img_color = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            edges = cv2.adaptiveThreshold(
                cv2.medianBlur(gray, 7), 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY, blockSize=9, C=2
            )
            cartoon = cv2.bitwise_and(img_color, img_color, mask=edges)
        elif cartoon_style == 'pencil':
            gray, sketch = cv2.pencilSketch(img, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
            cartoon = sketch
        elif cartoon_style == 'oil_paint':
            cartoon = cv2.xphoto.oilPainting(img, 7, 1)

        return cartoon
    except Exception as e:
        print(f"Error processing the image: {e}")
        return None

def load_font(size):
    """Load font with fallback."""
    try:
        font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
        return ImageFont.truetype(font_path, size)
    except:
        return ImageFont.load_default()

def wrap_text(text, font, max_width):
    """Wrap text to fit within the given width."""
    lines = []
    for line in text.split('\n'):
        lines.extend(textwrap.wrap(line, width=max_width))
    return lines

def generate_qr_code(data, size=100, fill_color="black", back_color="white"):
    """Generate a QR code with customizable colors."""
    qr = qrcode.QRCode(border=1)
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color=fill_color, back_color=back_color).resize((size, size))
    return img

def create_background(width, height, pattern='circles', color_scheme='random'):
    """Generate a dynamic background pattern."""
    bg = Image.new('RGB', (width, height), (255, 255, 255))
    draw = ImageDraw.Draw(bg)

    if color_scheme == 'random':
        get_color = lambda: tuple(random.randint(100, 255) for _ in range(3))
    else:
        get_color = lambda: tuple(random.choice([(255, 0, 0), (0, 255, 0), (0, 0, 255)]))

    if pattern == 'circles':
        for _ in range(50):
            x, y = random.randint(0, width), random.randint(0, height)
            radius = random.randint(20, 50)
            draw.ellipse([x - radius, y - radius, x + radius, y + radius], fill=get_color())
    elif pattern == 'squares':
        for _ in range(50):
            x, y = random.randint(0, width), random.randint(0, height)
            size = random.randint(20, 50)
            draw.rectangle([x, y, x + size, y + size], fill=get_color())

    return bg

def save_output(cartoon_img, bio, name, text_color='black', bg_pattern='circles'):
    """Save the final profile image with a cartoon, bio, and QR code."""
    if cartoon_img is None:
        print("Failed to generate cartoon image.")
        return

    width, height = cartoon_img.shape[1] + 400, max(cartoon_img.shape[0], 600)
    bg = create_background(width, height, pattern=bg_pattern)

    cartoon_pil = Image.fromarray(cv2.cvtColor(cartoon_img, cv2.COLOR_BGR2RGB))
    bg.paste(cartoon_pil, (0, 0))

    draw = ImageDraw.Draw(bg)
    font = load_font(22)
    text_position = (cartoon_img.shape[1] + 20, 30)

    wrapped_text = wrap_text(bio, font, 35)
    y_offset = text_position[1]

    for line in wrapped_text:
        text_width, text_height = font.getbbox(line)[2:4]  # Use getbbox correctly
        draw.text((text_position[0], y_offset), line, font=font, fill=text_color)
        y_offset += text_height + 10  # Adjust line spacing

    qr_code = generate_qr_code(bio, fill_color="darkblue", back_color="white")
    bg.paste(qr_code, (width - 120, height - 120))

    border_width = 5
    draw.rectangle([0, 0, cartoon_pil.width, cartoon_pil.height], outline=(0, 0, 0), width=border_width)

    output_path = f"{name.replace(' ', '_').lower()}_profile.png"
    bg.save(output_path)
    print(f"Profile saved as '{output_path}'")
    return output_path

def show_progress(task, total):
    """Display progress using tqdm."""
    for _ in tqdm(range(total), desc=task):
        pass

if __name__ == "__main__":
    image_path = '/content/Screenshot 2024-10-18 215335.png'

    bio, name = generate_character()

    show_progress("Cartoonifying Image", total=5)
    cartoon_img = cartoonify_image(image_path, cartoon_style='basic')

    show_progress("Saving Output", total=3)
    save_output(cartoon_img, bio, name, text_color='darkgreen', bg_pattern='squares')
